# NF26/AI07 - TD3 : Gestion des valeurs Atypiques/Manquantes (Correction)


In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

Pour résoudre le problème lié à "import pyspark.pandas as ps" :

Proposition par Chloe :

In [2]:
#!pip install numpy==1.23.5

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

Autre solution proposée par Tidiane :

In [4]:
# import pandas as pd
# import numpy as np
# from datetime import datetime, date
# np.NaN = np.nan
# np.string_ = np.bytes_
# np.float_ = np.float64
# np.unicode_ = np.str_

# import pyspark.pandas as ps
# from pyspark.sql import SparkSession
# from pyspark.sql import Row

In [5]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 19:01:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Choix du fuseau horaire (Solution trouvée par Thomas F.) :
spark.conf.set("spark.sql.session.timeZone", "CET")

## Introduction

L'objectif de ce TD est d'implémenter un ETL visant à gérer les données atypiques ou manquantes afin d'alimenter le modèle Étoile étudié lors du TD2. Pour rappel, les 6 diffrérentes bases de données transactionnelles données sont :
- `data_administration`: Ensemble des données administratives liées à chaque consultation.
- `data_medecins`: Ensemble des données liées aux informations associées à chaque Médecin.
- `data_diagnostics`: Ensemble des données liées aux diagnostic de chaque patient lors de chaque consultation.
- `data_treatments`: Ensemble des données liées aux descriptions de chaque traitement.
- `data_medicaments`: Ensemble des informations caractérisant chaque Médicament.
- `data_chambres`: Ensemble des informations caractérisant chaque Chambre de l'Hôpital.

On souhaite notamment pouvoir répondre aux requêtes suivantes : 
- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont accueilli des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période ?*".


## Lecture des données

Proposition de Code par Arman S-M afin de charger un dossier facilement sur Google Colab :

In [7]:
# Proposition de Code par Arman S-M
import zipfile

def unzip_data(filename):
    #Unzips filename into the current working directory.
    #Args:
    #filename (str): a filepath to a target zip folder to be unzipped.
    zip_ref = zipfile.ZipFile(filename, "r")
    zip_ref.extractall()
    zip_ref.close()

# Usage : unzip_data("nf26-td2.zip")

**Les cellules ci-dessous permettent de lire les différentes bases de données transactionnelles en format `pyspark.pandas.frame.DataFrame`.**

In [8]:
psdf_administration = ps.read_csv('data_administration.csv', index_col='KeyConsult')
psdf_administration.head(5)

,KeyPatient,Name,FirstName,NumSecu,Date_In,Pathology,KeyChambre,ChambreNumber,Date_Out
KeyConsult,,,,,,,,,
KeyConsult45056,KeyPatient38017,Name38017,FistName38017,NS45057,2023-11-23,Pathology27,KeyChambre37,C037,2023-11-24
KeyConsult45057,KeyPatient75131,Name75131,FistName75131,NS45058,2023-11-23,Pathology57,NULL,NULL,2023-11-23
KeyConsult45058,KeyPatient27733,Name27733,FistName27733,NS45059,2023-11-23,Pathology60,KeyChambre5,C005,2023-11-24
KeyConsult45059,KeyPatient76784,Name76784,FistName76784,NS45060,2023-11-23,Pathology30,KeyChambre43,C043,2023-11-25
KeyConsult45060,KeyPatient10645,Name10645,FistName10645,NS45061,2023-11-23,Pathology99,KeyChambre43,C043,2023-12-07


In [9]:
psdf_medecins = ps.read_csv('data_medecins.csv', index_col='KeyMedecin')
psdf_medecins.head(5)

,NameMedecin,FirstNameMedecin,SpecialityMedecin
KeyMedecin,,,
keyMedecin16,NameMedecin16,FirstNameMedecin16,Generaliste
keyMedecin17,NameMedecin17,FirstNameMedecin17,Generaliste
keyMedecin18,NameMedecin18,FirstNameMedecin18,Generaliste
keyMedecin19,NameMedecin19,FirstNameMedecin19,Chirurgien
keyMedecin6,NameMedecin6,FirstNameMedecin6,Dermatologiste


In [10]:
psdf_diagnostics = ps.read_csv('data_diagnostics.csv', index_col='KeyConsult')
psdf_diagnostics.head(5)

,KeyMedecin,KeyPatient,NamePatient,FirstNamePatient,NumSecu,Age,Weight,Temperature,Tension,Diabete,Pathology,KeyTreatment
KeyConsult,,,,,,,,,,,,
KeyConsult45056,keyMedecin5,KeyPatient38017,Name38017,FistName38017,NS45057,20.0,59.793081,37.569814,79.768608,0,Pathology27,KeyTreatment45056
KeyConsult45057,keyMedecin23,KeyPatient75131,Name75131,FistName75131,NS45058,15.0,61.467987,37.565551,164.781518,0,Pathology57,KeyTreatment45057
KeyConsult45058,keyMedecin10,KeyPatient27733,Name27733,FistName27733,NS45059,18.0,59.421285,37.481894,124.363853,0,Pathology60,KeyTreatment45058
KeyConsult45059,keyMedecin15,KeyPatient76784,Name76784,FistName76784,NS45060,20.0,57.974723,37.537167,98.881029,0,Pathology30,KeyTreatment45059
KeyConsult45060,keyMedecin29,KeyPatient10645,Name10645,FistName10645,NS45061,42.0,71.273588,40.379072,80.799002,1,Pathology99,KeyTreatment45060


In [11]:
psdf_treatments = ps.read_csv('data_treatments.csv', index_col='KeyTreatment')
psdf_treatments.head(5)

,KeyMedicament,QuantityMedicament
KeyTreatment,,
KeyTreatment20428,KeyMed2,2
KeyTreatment20429,KeyMed449,2
KeyTreatment20429,KeyMed321,3
KeyTreatment20429,KeyMed361,3
KeyTreatment20430,KeyMed505,1


In [12]:
psdf_medicaments = ps.read_csv('data_medicaments.csv', index_col='KeyMedicament')
psdf_medicaments.head(5)

,NameMedicament
KeyMedicament,
KeyMed913,Medicament913
KeyMed914,Medicament914
KeyMed915,Medicament915
KeyMed916,Medicament916
KeyMed917,Medicament917


In [13]:
psdf_chambres = ps.read_csv('data_chambres.csv', index_col='KeyChambre')
psdf_chambres.head(5)

,ChambreNumber
KeyChambre,
KeyChambre89,C089
KeyChambre90,C090
KeyChambre91,C091
KeyChambre92,C092
KeyChambre93,C093


Afin de pouvoir réaliser notre objectif, nous pourrons convertir ces données en format `pyspark.sql.dataframe.DataFrame`.

In [14]:
sdf_administration = psdf_administration.to_spark(index_col='KeyConsult')
sdf_medecins = psdf_medecins.to_spark(index_col='KeyMedecin')
sdf_diagnostics = psdf_diagnostics.to_spark(index_col='KeyConsult')
sdf_treatments = psdf_treatments.to_spark(index_col='KeyTreatment')
sdf_medicaments = psdf_medicaments.to_spark(index_col='KeyMedicament')
sdf_chambres = psdf_chambres.to_spark(index_col='KeyChambre')

In [15]:
from pyspark.sql.functions import *

## Exercices

On souhaite créer un modèle en étoile permettant de procéder facilement les requêtes suivantes : 
- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine Pathology durant une certaine période ?*".

Pour essayer de répondre à toutes ces questions, nous allons considérer le modèle étoile étudié lors du précédent TD.

![title](Schemas_Etoile_TD_new.pdf)

**Question 1.** Reprendre les algorithmes du TD2 pour construire le modèle Étoile ci-dessus.

In [16]:
# Tables de Faits :
sdf_FAITS_1 = sdf_diagnostics.select('KeyConsult','KeyPatient','KeyMedecin','KeyTreatment').alias('sdf_FAITS_1')
sdf_FAITS_2 = sdf_FAITS_1.withColumn('KeyDate', sdf_FAITS_1.KeyConsult)
sdf_FAITS_consults = sdf_FAITS_2.join(sdf_administration, 'KeyConsult').select(sdf_FAITS_2["*"],sdf_administration.KeyChambre)

# Tables de dimension associée aux Patients :
sdf_dim_patients = sdf_diagnostics.select('KeyPatient','KeyConsult','NamePatient','FirstNamePatient','NumSecu','Age','Weight','Temperature','Tension','Diabete','Pathology').alias('sdf_dim_patients')

# Tables de dimension associée aux Dates :
sdf_dim_date1 = sdf_FAITS_consults.select('KeyDate').alias('sdf_dim_date1')
sdf_adm = sdf_administration.select('KeyConsult','Date_In','Date_Out').alias('sdf_adm')
sdf_dim_dates = sdf_dim_date1.alias('sdf_dim_date1').join(sdf_adm.alias('sdf_adm'), sdf_dim_date1.KeyDate==sdf_adm.KeyConsult).drop('KeyConsult')

# Tables de dimension associée aux Medecins :
sdf_dim_medecins = sdf_medecins.select('*')

# Tables de dimension associée aux Traitements :
sdf_dim_traitement1 = sdf_treatments.alias('sdf_dim_traitements')
sdf_dim_traitements = sdf_dim_traitement1.join(sdf_medicaments, sdf_dim_traitement1.KeyMedicament==sdf_medicaments.KeyMedicament).select(sdf_dim_traitement1["*"],sdf_medicaments["NameMedicament"])

# Tables de dimension associée aux Chambres :
sdf_dim_chambres = sdf_chambres.select('*')


**Question 2.** Calculer l'âge moyen des patients :
- Pour la `Pathology95` durant le mois de mars 2023 : avg(Age) = ?
- Pour la `Pathology18` durant le mois de mars 2023 : avg(Age) = ?
- Pour la `Pathology76` durant le mois de juillet 2023 : avg(Age) = ?

In [17]:
PathologyQuest = "Pathology95"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology95 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()


############################################################################

PathologyQuest = "Pathology18"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology18 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

############################################################################

PathologyQuest = "Pathology76"
YearQuest = 2023
MonthQuest = 7

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology76 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

Pathology95 :


+-----------------+
|         avg(Age)|
+-----------------+
|156.7948717948718|
+-----------------+

Pathology18 :


+--------+
|avg(Age)|
+--------+
|Infinity|
+--------+

Pathology76 :
+--------+
|avg(Age)|
+--------+
|Infinity|
+--------+



## Problématique :
Il semblerait qu'il y ait des erreurs au niveau de la variable `Âge`. Certaines données ont été non-répertoriées dans la base de données `diagnostic` et ont été remplacées par la valeur `Inf`. D'autres données sont anormalement elevées et apportent un biais très important. 

**Objectifs :** Notre objectif sera de construire un ETL qui permette d'être robuste face à ces problèmes au niveau de la variable `Âge`. Nous allons pour cela faire un travail de pré-traitement de données par approche supervisée.
- Pour la suite, nous considèrerons qu'un âge est anormalement élevé s'il est superieur à 150 ans. En cas d'âge superieur à 150 ans, nous considèreons cette valeur comme atypique et nous devrons alors remplacer cette valeur par une valeur cohérentes.
- Pour la suite, Nous remplacerons les valeurs `Inf` par une estimation cohérente de l'âge en fonction des autres variables.

Comme nous l'avons vu en CM, cette phase de nettoyage de données doit se faire à partir des données source et juste avant le transfert dans le Data Warehouse, c'est à dire au cours de l'ETL.

## Questions :

**Question 3.** Construire un DataFrame contenant l'ensemble des observations 'non-problématiques' dans la base de données source `psdf_diagnostics`. Nous appellerons ce DataFrame  `psdf_diagnostics_clean`.

In [18]:
sdf_diagnostics_clean = sdf_diagnostics.where(sdf_diagnostics.Age != 'Inf').where(sdf_diagnostics.Age<150)
sdf_diagnostics_clean.show(5)


+---------------+------------+---------------+-----------+----------------+-------+----+------------------+------------------+------------------+-------+-----------+-----------------+
|     KeyConsult|  KeyMedecin|     KeyPatient|NamePatient|FirstNamePatient|NumSecu| Age|            Weight|       Temperature|           Tension|Diabete|  Pathology|     KeyTreatment|
+---------------+------------+---------------+-----------+----------------+-------+----+------------------+------------------+------------------+-------+-----------+-----------------+
|KeyConsult45056| keyMedecin5|KeyPatient38017|  Name38017|   FistName38017|NS45057|20.0| 59.79308073160008| 37.56981386307886| 79.76860765451696|      0|Pathology27|KeyTreatment45056|
|KeyConsult45057|keyMedecin23|KeyPatient75131|  Name75131|   FistName75131|NS45058|15.0|61.467987016139254|37.565550636587425|164.78151824294875|      0|Pathology57|KeyTreatment45057|
|KeyConsult45058|keyMedecin10|KeyPatient27733|  Name27733|   FistName27733|NS450

**Question 4.** À partir du DataFrame `sdf_diagnostics_clean`, calibrer un modèle d'estimation de l'âge par qui permettra de remplacer les valeurs d'âge atypiques ou manquantes par la moyenne des âges issus du DataFrame `sdf_diagnostics_clean`.

In [19]:
def function_estim_age_mean(sdf_diagnostics_clean):
    pd_diagnistic_clean = sdf_diagnostics_clean.toPandas()
    estim_age_mean = np.mean(pd_diagnistic_clean['Age'])
    return estim_age_mean

In [20]:
estim_age_mean = function_estim_age_mean(sdf_diagnostics_clean)
estim_age_mean

31.96982776007422

**Question 5.** Construire une fonction qui permette de remplacer les valeurs d'âge atypiques ou manquantes d'un DataFrame source par la moyenne des âges calculée à partir de la fonction `function_estim_age_mean(psdf_diagnostics_clean)`. Nous appelerons cette nouvelle fonction `function_ETL_age_mean(psdf_diagnostics)`. Cette nouvelle fonction ressortira un nouveau DataFrame contenant l'ensemble des valeurs (celle cohérentes et celles incohérentes estimées et remplacées). Nous appelerons ce nouveau DataFrame `psdf_diagnostics_estim_mean`.

In [21]:
def function_ETL_age_mean(sdf_diagnostics, estim_age_mean):
    
    sdf_diagnostics_estim_mean = sdf_diagnostics.withColumn('Age', when(sdf_diagnostics['Age']>150, estim_age_mean).otherwise(sdf_diagnostics['Age']))
        
    return sdf_diagnostics_estim_mean


In [22]:
sdf_diagnostics_estim_mean = function_ETL_age_mean(sdf_diagnostics, estim_age_mean)
sdf_diagnostics_estim_mean.show(5)

+---------------+------------+---------------+-----------+----------------+-------+----+------------------+------------------+------------------+-------+-----------+-----------------+
|     KeyConsult|  KeyMedecin|     KeyPatient|NamePatient|FirstNamePatient|NumSecu| Age|            Weight|       Temperature|           Tension|Diabete|  Pathology|     KeyTreatment|
+---------------+------------+---------------+-----------+----------------+-------+----+------------------+------------------+------------------+-------+-----------+-----------------+
|KeyConsult45056| keyMedecin5|KeyPatient38017|  Name38017|   FistName38017|NS45057|20.0| 59.79308073160008| 37.56981386307886| 79.76860765451696|      0|Pathology27|KeyTreatment45056|
|KeyConsult45057|keyMedecin23|KeyPatient75131|  Name75131|   FistName75131|NS45058|15.0|61.467987016139254|37.565550636587425|164.78151824294875|      0|Pathology57|KeyTreatment45057|
|KeyConsult45058|keyMedecin10|KeyPatient27733|  Name27733|   FistName27733|NS450

**Question 5.bis** Reprendre les algorithmes de la Question 1 pour construire le modèle Étoile ci-dessus mais en considérant cette fois le DataFrame `psdf_diagnostics_estim_mean` puis calculer l'âge moyen corrigé des des patients qui ont eu la `Pathology95` puis la `Pathology18` durant le mois de mars 2023.

In [23]:
# Tables de Faits :
sdf_FAITS_1 = sdf_diagnostics_estim_mean.select('KeyConsult','KeyPatient','KeyMedecin','KeyTreatment').alias('sdf_FAITS_1')
sdf_FAITS_2 = sdf_FAITS_1.withColumn('KeyDate', sdf_FAITS_1.KeyConsult)
sdf_FAITS_consults = sdf_FAITS_2.join(sdf_administration, 'KeyConsult').select(sdf_FAITS_2["*"],sdf_administration.KeyChambre)

# Tables de dimension associée aux Patients :
sdf_dim_patients = sdf_diagnostics_estim_mean.select('KeyPatient','KeyConsult','NamePatient','FirstNamePatient','NumSecu','Age','Weight','Temperature','Tension','Diabete','Pathology').alias('sdf_dim_patients')

# Tables de dimension associée aux Dates :
sdf_dim_date1 = sdf_FAITS_consults.select('KeyDate').alias('sdf_dim_date1')
sdf_adm = sdf_administration.select('KeyConsult','Date_In','Date_Out').alias('sdf_adm')
sdf_dim_dates = sdf_dim_date1.alias('sdf_dim_date1').join(sdf_adm.alias('sdf_adm'), sdf_dim_date1.KeyDate==sdf_adm.KeyConsult).drop('KeyConsult')

# Tables de dimension associée aux Medecins :
sdf_dim_medecins = sdf_medecins.select('*')

# Tables de dimension associée aux Traitements :
sdf_dim_traitement1 = sdf_treatments.alias('sdf_dim_traitements')
sdf_dim_traitements = sdf_dim_traitement1.join(sdf_medicaments, sdf_dim_traitement1.KeyMedicament==sdf_medicaments.KeyMedicament).select(sdf_dim_traitement1["*"],sdf_medicaments["NameMedicament"])

# Tables de dimension associée aux Chambres :
sdf_dim_chambres = sdf_chambres.select('*')


############################################################################

PathologyQuest = "Pathology95"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology95 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()


############################################################################

PathologyQuest = "Pathology18"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology18 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

############################################################################

PathologyQuest = "Pathology76"
YearQuest = 2023
MonthQuest = 7

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology76 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

Pathology95 :


+-----------------+
|         avg(Age)|
+-----------------+
|40.48563219282432|
+-----------------+

Pathology18 :


+------------------+
|          avg(Age)|
+------------------+
|20.179396555201482|
+------------------+

Pathology76 :
+-----------------+
|         avg(Age)|
+-----------------+
|69.34131125684405|
+-----------------+



**Question 6.** À partir du DataFrame `psdf_diagnostics_clean`, calibrer un modèle d'estimation de l'âge par qui permettra de remplacer les valeurs d'âge atypiques ou manquantes par apprentissage supervisé à partir du DataFrame `psdf_diagnostics_clean`. Nous pourrons pour cela utiliser les fonctions proposées par Scikit-Learn : https://scikit-learn.org/stable/

Premier exemple avec les variables "weight", "temperature", "tension" :

In [24]:


#Construction XTrain/Ytrain :
XTrain_sdf = sdf_diagnostics_clean.select('Weight','Temperature','Tension')
XTrain = XTrain_sdf.toPandas()
YTrain_sdf = sdf_diagnostics_clean.select('Age')
YTrain = YTrain_sdf.toPandas()

# Apprentissage model lineaire :
from sklearn import linear_model
reg_model = linear_model.LinearRegression()
reg_model.fit(XTrain, YTrain)

# Estimation des valeurs manquantes/atypiques :
df = sdf_diagnostics.toPandas()
indexToPredict = df[df['Age']>150].index
XToPredict = df.loc[indexToPredict, ['Weight','Temperature','Tension']]
YAgeEstim = reg_model.predict(XToPredict)
df.loc[indexToPredict, 'Age'] = YAgeEstim

sdf_diagnostics_estim_ML = spark.createDataFrame(df)




In [25]:
# Tables de Faits :
sdf_FAITS_1 = sdf_diagnostics_estim_ML.select('KeyConsult','KeyPatient','KeyMedecin','KeyTreatment').alias('sdf_FAITS_1')
sdf_FAITS_2 = sdf_FAITS_1.withColumn('KeyDate', sdf_FAITS_1.KeyConsult)
sdf_FAITS_consults = sdf_FAITS_2.join(sdf_administration, 'KeyConsult').select(sdf_FAITS_2["*"],sdf_administration.KeyChambre)

# Tables de dimension associée aux Patients :
sdf_dim_patients = sdf_diagnostics_estim_ML.select('KeyPatient','KeyConsult','NamePatient','FirstNamePatient','NumSecu','Age','Weight','Temperature','Tension','Diabete','Pathology').alias('sdf_dim_patients')

# Tables de dimension associée aux Dates :
sdf_dim_date1 = sdf_FAITS_consults.select('KeyDate').alias('sdf_dim_date1')
sdf_adm = sdf_administration.select('KeyConsult','Date_In','Date_Out').alias('sdf_adm')
sdf_dim_dates = sdf_dim_date1.alias('sdf_dim_date1').join(sdf_adm.alias('sdf_adm'), sdf_dim_date1.KeyDate==sdf_adm.KeyConsult).drop('KeyConsult')

# Tables de dimension associée aux Medecins :
sdf_dim_medecins = sdf_medecins.select('*')

# Tables de dimension associée aux Traitements :
sdf_dim_traitement1 = sdf_treatments.alias('sdf_dim_traitements')
sdf_dim_traitements = sdf_dim_traitement1.join(sdf_medicaments, sdf_dim_traitement1.KeyMedicament==sdf_medicaments.KeyMedicament).select(sdf_dim_traitement1["*"],sdf_medicaments["NameMedicament"])

# Tables de dimension associée aux Chambres :
sdf_dim_chambres = sdf_chambres.select('*')


############################################################################

PathologyQuest = "Pathology95"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology95 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()


############################################################################

PathologyQuest = "Pathology18"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology18 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

############################################################################

PathologyQuest = "Pathology76"
YearQuest = 2023
MonthQuest = 7

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology76 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

Pathology95 :


+-----------------+
|         avg(Age)|
+-----------------+
|41.79529497105525|
+-----------------+

Pathology18 :


+-----------------+
|         avg(Age)|
+-----------------+
|19.89145966212464|
+-----------------+

Pathology76 :


+-----------------+
|         avg(Age)|
+-----------------+
|69.94312060593299|
+-----------------+



Deuxième exemple en ajoutant la variable `Pathology` en ***one-hot encoding*** ainsi que la variable `Diabete` :

In [26]:
def function_encode_dfWithPathology(Xdf):
    pathology_list = []
    for i in range(100):
        pathology_list.append('Pathology' + str(i+1))
    pathology_df = pd.DataFrame(pathology_list)
    pathology_df.columns = ['Pathology']
    
    indexes = Xdf.index

    X = np.zeros((np.array(Xdf).shape[0], np.array(Xdf).shape[1]+pathology_df.shape[0]-1))
    X[:,0:3] = np.array(Xdf)[:,0:3]
    for i in range(X.shape[0]):
        j = pathology_df.index[pathology_df['Pathology'] == Xdf.loc[indexes[i],'Pathology']].tolist()[0]
        X[i,4+j] = 1

    Xnew = pd.DataFrame(X)
    Xnew.columns = ['Weight','Temperature','Tension','Diabete'] + pathology_list

    return Xnew

In [27]:
def function_estim_age_ML(sdf_diagnostics_clean):
    
    YTrain = sdf_diagnostics_clean.select('Age').toPandas()
    Xdf = sdf_diagnostics_clean.select('Weight','Temperature','Tension','Diabete','Pathology').toPandas()
    XTrain = function_encode_dfWithPathology(Xdf)
    
    # Apprentissage model lineaire :
    from sklearn import linear_model
    reg_model = linear_model.LinearRegression()
    reg_model.fit(XTrain, YTrain)
    
    return reg_model

In [28]:
reg_model = function_estim_age_ML(sdf_diagnostics_clean)

**Question 8.** Construire une fonction qui permette de remplacer les valeurs d'âge atypiques ou manquantes d'un DataFrame source par la valeurs estimée des âges à partir de la fonction `function_estim_age_ML(psdf_diagnostics_clean)`. Nous appelerons cette nouvelle fonction `function_ETL_age_ML(psdf_diagnostics)`. Cette nouvelle fonction ressortira un nouveau DataFrame contenant l'ensemble des valeurs (celle cohérentes et celles incohérentes estimées puis remplacées). Nous appelerons ce nouveau DataFrame `psdf_diagnostics_estim_ML`.

In [29]:
def function_ETL_age_ML(sdf_diagnostics, reg_model):
    
    df = sdf_diagnostics.toPandas()
    indexToPredict = df[df['Age']>150].index
    XToPredict = df.loc[indexToPredict, ['Weight','Temperature','Tension','Diabete','Pathology']]
    XToPredict_encoded = function_encode_dfWithPathology(XToPredict)
    YAgeEstim = reg_model.predict(XToPredict_encoded)
    df.loc[indexToPredict, 'Age'] = YAgeEstim
    
    sdf_diagnostics_estim_ML = spark.createDataFrame(df)
 
    return sdf_diagnostics_estim_ML

In [30]:
sdf_diagnostics_estim_ML = function_ETL_age_ML(sdf_diagnostics, reg_model)
#sdf_diagnostics_estim_ML.show(10)

**Question 9.** Reprendre les algorithmes de la Question 1 pour construire le modèle Étoile ci-dessus mais en considérant cette fois le DataFrame `psdf_diagnostics_estim_ML` puis calculer l'âge moyen corrigé des des patients qui ont eu la `Pathology95` puis la `Pathology18` durant le mois de mars 2023.

In [31]:
# Tables de Faits :
sdf_FAITS_1 = sdf_diagnostics_estim_ML.select('KeyConsult','KeyPatient','KeyMedecin','KeyTreatment').alias('sdf_FAITS_1')
sdf_FAITS_2 = sdf_FAITS_1.withColumn('KeyDate', sdf_FAITS_1.KeyConsult)
sdf_FAITS_consults = sdf_FAITS_2.join(sdf_administration, 'KeyConsult').select(sdf_FAITS_2["*"],sdf_administration.KeyChambre)

# Tables de dimension associée aux Patients :
sdf_dim_patients = sdf_diagnostics_estim_ML.select('KeyPatient','KeyConsult','NamePatient','FirstNamePatient','NumSecu','Age','Weight','Temperature','Tension','Diabete','Pathology').alias('sdf_dim_patients')

# Tables de dimension associée aux Dates :
sdf_dim_date1 = sdf_FAITS_consults.select('KeyDate').alias('sdf_dim_date1')
sdf_adm = sdf_administration.select('KeyConsult','Date_In','Date_Out').alias('sdf_adm')
sdf_dim_dates = sdf_dim_date1.alias('sdf_dim_date1').join(sdf_adm.alias('sdf_adm'), sdf_dim_date1.KeyDate==sdf_adm.KeyConsult).drop('KeyConsult')

# Tables de dimension associée aux Medecins :
sdf_dim_medecins = sdf_medecins.select('*')

# Tables de dimension associée aux Traitements :
sdf_dim_traitement1 = sdf_treatments.alias('sdf_dim_traitements')
sdf_dim_traitements = sdf_dim_traitement1.join(sdf_medicaments, sdf_dim_traitement1.KeyMedicament==sdf_medicaments.KeyMedicament).select(sdf_dim_traitement1["*"],sdf_medicaments["NameMedicament"])

# Tables de dimension associée aux Chambres :
sdf_dim_chambres = sdf_chambres.select('*')


############################################################################

PathologyQuest = "Pathology95"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology95 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()


############################################################################

PathologyQuest = "Pathology18"
YearQuest = 2023
MonthQuest = 3

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology18 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

############################################################################

PathologyQuest = "Pathology76"
YearQuest = 2023
MonthQuest = 7

df_DatesQuest = sdf_dim_dates.filter((year("Date_in")==YearQuest) & (month("Date_in")==MonthQuest)).select(col("KeyDate"))
df_PathologyQuest = sdf_dim_patients.filter(col("Pathology")==PathologyQuest)

FactDatesQuest = sdf_FAITS_consults.alias("t1").join(df_DatesQuest.alias("t2"),'KeyDate').select(sdf_FAITS_consults["*"])
PatientsObjectifQuest = df_PathologyQuest.alias("s1").join(FactDatesQuest.alias("s2"),'KeyConsult').select(df_PathologyQuest["*"])
print("Pathology76 :")
PatientsObjectifQuest.select(avg(col("Age"))).show()

Pathology95 :


+-----------------+
|         avg(Age)|
+-----------------+
|40.90808886441832|
+-----------------+

Pathology18 :


+------------------+
|          avg(Age)|
+------------------+
|19.940177996696438|
+------------------+

Pathology76 :


+-----------------+
|         avg(Age)|
+-----------------+
|70.33922898509589|
+-----------------+



### Remarque. Dans le but de pouvoir comparer les résultats pour chaque méthode, les réponses que nous aurions eu à partir des données initialement clean et réelles auraient été :
- Pour la `Pathology95` durant le mois de mars 2023 : avg(Age) = 41.205128205128204
- Pour la `Pathology18` durant le mois de mars 2023 : avg(Age) = 20.0
- Pour la `Pathology76` durant le mois de juillet 2023 : avg(Age) = 70.56756756756756